In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sklearn
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv('CityMaster1.4.csv')

In [3]:
# for columns in df:
#         print(columns)

In [4]:
df = df.drop(columns={'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0_y', 'STATE_y', 'COUNTY_y'})

In [5]:
df.select_dtypes(exclude=[np.number]).columns

Index(['Legal Status', 'Medicinal', 'Decriminalized', 'time_zone',
       'PresW_2012', 'PresW_2016', 'PresW_2020', 'MSA', 'violent_crime',
       'property_crime', 'burglary', 'larceny_theft', 'city', 'county_name'],
      dtype='object')

In [6]:
df_app = df.copy()

In [7]:
# note: drop first dummy for PCA, but not for use in web app

In [8]:
df.head()

,Population_estimate,FIPS,Acreage,City_Pop,HHINCOME_mean,HHINCOME_median,VALUEH_mean,VALUEH_median,AGE_mean,AGE_median,...,st_local_gen_sales_tax,realestate_tax,personal_prop_tax,total_taxes_paid,total_tax_payments_amt,FIPS_2digit,City_Dummy,new_city_pop,city,county_name
0,NaN,1000,NaN,1970.0,668874.8511,74805.0,2807735.970,270000.0,43.800921,45.0,...,45611.0,334941.0,46903.0,974173.0,24008761.0,1,1,NaN,NaN,NaN
1,NaN,1001,380219.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,688.0,1804.0,303.0,8741.0,228794.0,1,0,38124.0,Prattville,Autauga
2,246435.0,1003,1016235.68,0.0,693232.7108,90000.0,2359146.832,345000.0,46.806620,51.0,...,3321.0,19336.0,2038.0,58098.0,1359923.0,1,0,27861.0,Summerdale,Baldwin
3,NaN,1005,564523.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,127.0,518.0,87.0,1903.0,66115.0,1,0,12746.0,Louisville,Barbour
4,NaN,1007,397309.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,115.0,257.0,170.0,1770.0,87801.0,1,0,2994.0,Woodstock,Bibb


In [9]:
# List of columns to convert to dummies
columns_to_dummy = ['Legal Status', 'Medicinal', 'Decriminalized', 'time_zone', 'PresW_2012', 'PresW_2016', 'PresW_2020']

# Create dummy variables for the specified columns and drop the original columns
df = pd.get_dummies(df, columns=columns_to_dummy, drop_first=True)

df = df.drop(columns={'IND_Mil'}) # drop first equ

In [10]:
df = df.drop(columns={'MSA', 'city', 'county_name', 'FIPS'})
# attempt to convert to numeric for the following columns, drop values that are not numeric:
# 'violent_crime', 'property_crime', 'burglary', 'larceny_theft'

In [11]:
df['violent_crime'].value_counts()

violent_crime
174.2    4
44.3     4
200.5    4
114.6    2
471.7    2
        ..
585.7    1
305      1
692.1    1
215.9    1
326.8    1
Name: count, Length: 182, dtype: int64

In [12]:
# Assuming df is your DataFrame and columns_to_convert is your list of columns
columns_to_convert = ['violent_crime', 'property_crime', 'burglary', 'larceny_theft']

for column in columns_to_convert:
    # Convert column to numeric, coercing errors to NaN
    df[column] = pd.to_numeric(df[column], errors='coerce')

In [13]:
# for every column in df, replace False with 0 and True with 1
for column in df.columns:
    # Check if the column's data type is boolean
    if df[column].dtype == bool:
        # Replace True with 1 and False with 0
        df[column] = df[column].astype(int)

df.head()

,Population_estimate,Acreage,City_Pop,HHINCOME_mean,HHINCOME_median,VALUEH_mean,VALUEH_median,AGE_mean,AGE_median,Female_Rate,...,time_zone_M,time_zone_MC,time_zone_MP,time_zone_P,time_zone_S,time_zone_V,time_zone_e,PresW_2012_REPUBLICAN,PresW_2016_REPUBLICAN,PresW_2020_REPUBLICAN
0,NaN,NaN,1970.0,668874.8511,74805.0,2807735.970,270000.0,43.800921,45.0,0.515109,...,0,0,0,0,0,0,0,0,0,0
1,NaN,380219.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,1,1
2,246435.0,1016235.68,0.0,693232.7108,90000.0,2359146.832,345000.0,46.806620,51.0,0.505807,...,0,0,0,0,0,0,0,1,1,1
3,NaN,564523.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,1,1
4,NaN,397309.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,1,1,1


In [14]:
# df.to_csv('df_for_PCA.csv')

In [15]:
# List of columns to convert to dummies
columns_to_dummy = ['Legal Status', 'Medicinal', 'Decriminalized', 'time_zone', 'PresW_2012', 'PresW_2016', 'PresW_2020']

# Create dummy variables for the specified columns and drop the original columns
df_app = pd.get_dummies(df_app, columns=columns_to_dummy, drop_first=False)

# no IND drop

In [16]:
columns_to_convert = ['violent_crime', 'property_crime', 'burglary', 'larceny_theft']


for column in columns_to_convert:

    # Convert column to numeric, coercing errors to NaN

    df_app[column] = pd.to_numeric(df_app[column], errors='coerce')

# for every column in df, replace False with 0 and True with 1

for column in df_app.columns:

    # Check if the column's data type is boolean

    if df_app[column].dtype == bool:

        # Replace True with 1 and False with 0

        df_app[column] = df_app[column].astype(int)

df_app.head()

,Population_estimate,FIPS,Acreage,City_Pop,HHINCOME_mean,HHINCOME_median,VALUEH_mean,VALUEH_median,AGE_mean,AGE_median,...,time_zone_P,time_zone_S,time_zone_V,time_zone_e,PresW_2012_DEMOCRAT,PresW_2012_REPUBLICAN,PresW_2016_DEMOCRAT,PresW_2016_REPUBLICAN,PresW_2020_DEMOCRAT,PresW_2020_REPUBLICAN
0,NaN,1000,NaN,1970.0,668874.8511,74805.0,2807735.970,270000.0,43.800921,45.0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,1001,380219.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,0,1,0,1
2,246435.0,1003,1016235.68,0.0,693232.7108,90000.0,2359146.832,345000.0,46.806620,51.0,...,0,0,0,0,0,1,0,1,0,1
3,NaN,1005,564523.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,0,0,1,0,1
4,NaN,1007,397309.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,0,1,0,1


In [17]:
for columns in df_app:
    print(columns)

Population_estimate
FIPS
Acreage
City_Pop
HHINCOME_mean
HHINCOME_median
VALUEH_mean
VALUEH_median
AGE_mean
AGE_median
Female_Rate
Employment_Rate
Unemployment_Rate
NotLaborForce_Rate
IND_Arts_Ent_Rec_Food
IND_Constr
IND_Fin_Ins_RealEstate
IND_Info
IND_Manu
IND_Mil
IND_OtherServices
IND_PublicAdmin
IND_Retail
IND_Transpo_Ware_Util
IND_Wholesale
Adherents as % of Population
Congregations Per 100,000 Pop. Rank
population
households
alltransit_performance_score
Male_Rate
Cropland_Rate
Pasture_Rate
Forest_Rate
Density_Score
Density_Index
Population_Density
incident_counts
Arts_Ent_Rec_Est
Food_Est
Annual_Precip
Annual_Snowfall
Winter_Temp
Summer_Temp
HS_Grad_num
HS_GRAD_RATE
MSA
violent_crime
murder_non-neg_manslaughter
rape
robbery
aggravated_assault
property_crime
burglary
larceny_theft
motor_vehicle_theft
YEAR_Crime
Num_Hospitals
Hospital overall rating
Num_Returns
adj_gross_income
total_income
st_local_income_tax
st_local_gen_sales_tax
realestate_tax
personal_prop_tax
total_taxes_paid
t

In [18]:
# final data handling before norm/inv

df_app['Density'] = df_app['households'] / df_app['Acreage']

In [19]:
df_app[['total_taxes_paid', 'total_tax_payments_amt']].head()

,total_taxes_paid,total_tax_payments_amt
0,974173.0,24008761.0
1,8741.0,228794.0
2,58098.0,1359923.0
3,1903.0,66115.0
4,1770.0,87801.0


In [20]:
# Summing Republican wins
df_app['Rep_W'] = df_app['PresW_2012_REPUBLICAN'] + df_app['PresW_2016_REPUBLICAN'] + df_app['PresW_2020_REPUBLICAN']

# Summing Democratic wins
df_app['Dem_W'] = (1 - df_app['PresW_2012_REPUBLICAN']) + (1 - df_app['PresW_2016_REPUBLICAN']) + (1 - df_app['PresW_2020_REPUBLICAN'])

In [21]:
df_app['Average Tax Burden'] = df_app['total_tax_payments_amt']/df_app['new_city_pop']
# df_app['Average Tax Burden'].describe() # makes no sense but it'll be normalized (shrug emoji)

In [22]:
df_app['theft'] = df_app[['robbery', 'property_crime', 'burglary', 'larceny_theft', 'motor_vehicle_theft']].sum(axis=1)

In [23]:
# time zones must be updated
columns = [
    "time_zone_C", "time_zone_CE", "time_zone_CM", "time_zone_E", "time_zone_F",
    "time_zone_G", "time_zone_H", "time_zone_J", "time_zone_K", "time_zone_M",
    "time_zone_MC", "time_zone_MP", "time_zone_P", "time_zone_S", "time_zone_V",
    "time_zone_e"
]

# # Initialize a dictionary to store counts
# time_zone_counts = {}

# # Calculate counts of 1s for each time zone column and store in the dictionary
# for column in columns:
#     count_of_ones = df_app[column].value_counts().get(1, 0)
#     time_zone_counts[column] = count_of_ones

# # Create a list from the dictionary, sorted by the counts in descending order
# sorted_time_zones = sorted(time_zone_counts.items(), key=lambda item: item[1], reverse=True)

# # Print the sorted time zones with their counts
# for zone, count in sorted_time_zones:
#     print(f"{zone}: {count}")

# out of columns, keep _E, _C, _M, _P -- all others create time_zone_other dummy

import pandas as pd

# Columns to keep without changes or specific checks
keep_columns = [
    "time_zone_E", "time_zone_C", "time_zone_M", "time_zone_P"
]

# Initialize a new column for 'other' time zones to accumulate counts of 1s
df_app['time_zone_other'] = 0

# Iterate through columns and process
for column in columns:
    if column not in keep_columns:
        # Accumulate counts of ones in 'time_zone_other'
        df_app['time_zone_other'] += df_app[column].apply(lambda x: 1 if x == 1 else 0)

# Optionally, drop the other time zone columns if they are no longer needed
df_app.drop(columns=[col for col in columns if col not in keep_columns], inplace=True)

In [24]:
# onto normalization and inversion for df_app:


# vars to normalize (higher or lower = better) AND make df_app['type'] = norm
df_norm = [
    "HHINCOME_mean", "HHINCOME_median",
    "IND_Arts_Ent_Rec_Food", "IND_Constr", "IND_Fin_Ins_RealEstate", "IND_Info",
    "IND_Manu", "IND_OtherServices", "IND_PublicAdmin", "IND_Retail",
    "IND_Transpo_Ware_Util", "IND_Wholesale", "Adherents as % of Population",
    "Congregations Per 100,000 Pop. Rank", "Housing Density", "alltransit_performance_score",
    "Cropland_Rate", "Pasture_Rate", "Forest_Rate", "Arts_Ent_Rec_Est",
    "Food_Est", "HS_GRAD_RATE", "Num_Hospitals",
    "Hospital overall rating", "Average Tax Burden", "new_city_pop", "Legal Status_Fully Legal",
    "Decriminalized_Yes", "time_zone_E", "time_zone_M", "time_zone_P", "time_zone_C",
    "time_zone_other", "Dem_W", "Rep_W", "Acreage"
]


# vars to keep true values (for sliders) AND make df_app['type'] = gold
df_gold = [
    "VALUEH_median", "AGE_mean", "Annual_Precip",
    "Annual_Snowfall", "Winter_Temp", "Summer_Temp"
]


# vars to invert the normalization AND change df_app['type'] = inv_norm
df_inv = [
    "Unemployment_Rate", "incident_counts", "violent_crime", "theft", "Average Tax Burden"
]


# to drop
df_obsolete = [
    "Population_estimate", "City_Pop", "Female_Rate", "NotLaborForce_Rate",
    "population", "households", "Density_Score", "Density_Index",
    "Population_Density", "HS_Grad_num", "murder_non-neg_manslaughter", "rape",
    "robbery", "aggravated_assault", "property_crime", "burglary",
    "larceny_theft", "motor_vehicle_theft", "YEAR_Crime", "Num_Returns",
    "adj_gross_income", "total_income", "st_local_income_tax", "st_local_gen_sales_tax",
    "realestate_tax", "personal_prop_tax", "total_taxes_paid", "total_tax_payments_amt",
    "FIPS_2digit", "City_Dummy", "Legal Status_Mixed", "Medicinal_CBD Oil Only*",
    "Medicinal_No", "Medicinal_Yes", "PresW_2012_REPUBLICAN", "PresW_2016_REPUBLICAN",
    "PresW_2020_REPUBLICAN", "MSA", "Density", "Medicinal_CBD Oil Only", "Male_Rate",
    "Legal Status_Fully Illegal", "Decriminalized_No", "FIPS", "county_name", "IND_Mil", "PresW_2012_DEMOCRAT",
    "PresW_2016_DEMOCRAT", "PresW_2020_DEMOCRAT", "Employment_Rate"

]

In [25]:
df_app["city"].head()

0           NaN
1    Prattville
2    Summerdale
3    Louisville
4     Woodstock
Name: city, dtype: object

In [26]:
# Combining all variable lists into one set
all_listed_vars = set(df_norm + df_inv + df_gold + df_obsolete)

# Getting all columns from df_app
all_columns_set = set(df_app.columns)

# Finding columns in df_app not in any list
unlisted_columns = all_columns_set - all_listed_vars

# Printing unlisted columns
print("Unlisted columns in df_app:")
for column in unlisted_columns:
    print(column)

Unlisted columns in df_app:
city


In [40]:
# for columns in df_norm, normalize them
# Initialize the scaler
scaler = MinMaxScaler()

# Normalize columns listed in df_norm
for col in df_norm:
    if col in df_app.columns:
        df_app[col] = scaler.fit_transform(df_app[col].values.reshape(-1, 1))

# for columns in df_gold, do nothing

# for columns in df_inv, normalize then invert
# Normalize and then invert columns listed in df_inv
for col in df_inv:
    if col in df_app.columns:
        # Normalize
        normalized_data = scaler.fit_transform(df_app[col].values.reshape(-1, 1))
        # Invert
        df_app[col] = 1 - normalized_data


# for columns in df_obsolete, drop them
# Dropping obsolete columns safely
df_app.drop(columns=[col for col in df_obsolete if col in df_app.columns], inplace=True)

C:\Users\benja\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:479: RuntimeWarning: All-NaN slice encountered
  data_min = np.nanmin(X, axis=0)
C:\Users\benja\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:480: RuntimeWarning: All-NaN slice encountered
  data_max = np.nanmax(X, axis=0)


In [28]:
# Dictionary to hold type information
column_types = {}

# Set type for each column
for col in df_app.columns:
    if col in df_norm:
        column_types[col] = 'norm'
    elif col in df_inv:
        column_types[col] = 'inv_norm'
    elif col in df_gold:
        column_types[col] = 'gold'
    else:
        column_types[col] = 'key'

# Now, column_types dictionary holds the type of each column
print(column_types)

{'Acreage': 'norm', 'HHINCOME_mean': 'norm', 'HHINCOME_median': 'norm', 'VALUEH_mean': 'gold', 'VALUEH_median': 'gold', 'AGE_mean': 'gold', 'AGE_median': 'gold', 'Employment_Rate': 'norm', 'Unemployment_Rate': 'inv_norm', 'IND_Arts_Ent_Rec_Food': 'norm', 'IND_Constr': 'norm', 'IND_Fin_Ins_RealEstate': 'norm', 'IND_Info': 'norm', 'IND_Manu': 'norm', 'IND_OtherServices': 'norm', 'IND_PublicAdmin': 'norm', 'IND_Retail': 'norm', 'IND_Transpo_Ware_Util': 'norm', 'IND_Wholesale': 'norm', 'Adherents as % of Population': 'norm', 'Congregations Per 100,000 Pop. Rank': 'norm', 'alltransit_performance_score': 'norm', 'Cropland_Rate': 'norm', 'Pasture_Rate': 'norm', 'Forest_Rate': 'norm', 'incident_counts': 'inv_norm', 'Arts_Ent_Rec_Est': 'norm', 'Food_Est': 'norm', 'Annual_Precip': 'gold', 'Annual_Snowfall': 'gold', 'Winter_Temp': 'gold', 'Summer_Temp': 'gold', 'HS_GRAD_RATE': 'norm', 'violent_crime': 'inv_norm', 'Num_Hospitals': 'norm', 'Hospital overall rating': 'norm', 'new_city_pop': 'norm', 

In [43]:
{'Annual_Snowfall': 'Annual Snowfall', 'IND_Retail': 'The Retail Industry is Strong',
 'time_zone_E': 'The Time Zone is Eastern Time', 'IND_Info': 'The Information Industry is Strong',
 'IND_Manu': 'The Manufacturing Industry is Strong', 'violent_crime': 'Violent Crime is Low',
 'Congregations Per 100,000 Pop. Rank': 'There are Places of Worship Nearby',
 'Num_Hospitals': 'There are a Lot of Hospitals Nearby', 'time_zone_C': 'The Time Zone is Central Time',
 'Housing Density': 'You are in a Dense Area', 'HHINCOME_median': 'Median Income is High in the Area',
 'IND_PublicAdmin': 'The Public Admin Industry is Strong', 'Pasture_Rate': 'There is a Lot of Pasture Nearby',
 'Cropland_Rate': 'There is a Lot of Cropland Nearby', 'Adherents as % of Population': 'You are in a Highly Religious Area',
 'Rep_W': 'The Area Votes Republican', 'VALUEH_median': 'Home Value',
 'Forest_Rate': 'There is a Lot of Forest Nearby', 'new_city_pop': 'The City Population is High',
 'IND_Transpo_Ware_Util': 'The Transportation/Warehousing/Utilities Industry is Strong',
 'alltransit_performance_score': 'The City Has High-Quality Transit', 'theft': 'Theft is Low',
 'IND_Arts_Ent_Rec_Food': 'The Arts/Entertainment/Recreation/Food Industry is Strong',
 'Legal Status_Fully Legal': 'Marijuana is Legal', 'Summer_Temp': 'Summer Temperature',
 'time_zone_M': 'The Time Zone is Mountain Time', 'IND_Constr': 'The Construction Industry is Strong',
 'IND_Wholesale': 'The Wholesale Industry is Strong',
 'Arts_Ent_Rec_Est': 'There are a Lot of Entertainment and Recreation Establishments',
 'Average Tax Burden': 'Average Tax Burden is Low', 'Dem_W': 'The Area Votes Democrat',
 'Food_Est': 'There are a Lot of Restaurants', 'AGE_mean': 'Average Age',
 'IND_Fin_Ins_RealEstate': 'The Finance/Insurance/Real Estate Industry is Strong',
 'Hospital overall rating': 'Nearby Hospitals are High-Quality', 'HS_GRAD_RATE': 'High School Graduation Rate is High',
 'time_zone_P': 'The Time Zone is Pacific Time', 'IND_OtherServices': 'The "Other Services" Industry is Strong',
 'incident_counts': 'Natural Disaster Prevalence is Low', 'Unemployment_Rate': 'Unemployment Rate is Low',
 'Annual_Precip': 'Annual Rainfall', 'time_zone_other': 'The Time Zone is Not ET/CT/MT/PT',
 'Winter_Temp': 'Winter Temperature', 'Decriminalized_Yes': 'Marijuana is Decriminalized'}

{'Annual_Snowfall': 'Annual Snowfall',
 'IND_Retail': 'The Retail Industry is Strong',
 'time_zone_E': 'The Time Zone is Eastern Time',
 'IND_Info': 'The Information Industry is Strong',
 'IND_Manu': 'The Manufacturing Industry is Strong',
 'violent_crime': 'Violent Crime is Low',
 'Congregations Per 100,000 Pop. Rank': 'There are Places of Worship Nearby',
 'Num_Hospitals': 'There are a Lot of Hospitals Nearby',
 'time_zone_C': 'The Time Zone is Central Time',
 'Housing Density': 'You are in a Dense Area',
 'HHINCOME_median': 'Median Income is High in the Area',
 'IND_PublicAdmin': 'The Public Admin Industry is Strong',
 'Pasture_Rate': 'There is a Lot of Pasture Nearby',
 'Cropland_Rate': 'There is a Lot of Cropland Nearby',
 'Adherents as % of Population': 'You are in a Highly Religious Area',
 'Rep_W': 'The Area Votes Republican',
 'VALUEH_median': 'Home Value',
 'Forest_Rate': 'There is a Lot of Forest Nearby',
 'new_city_pop': 'The City Population is High',
 'IND_Transpo_Ware_Uti

In [36]:
df_app.shape

(3896, 49)

In [31]:
# df_app.to_csv('CityMaster1.5.csv')